# YouTube Agent with OpenAI Agents SDK
## ABB #4 - Session 4

Code authored by: Shaw Talebi

### imports

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
from agents import Agent, function_tool, Runner, ItemHelpers, RunContextWrapper
from openai.types.responses import ResponseTextDeltaEvent
from dotenv import load_dotenv
import asyncio

In [2]:
# import environment variables from .env file
load_dotenv()

True

### define instructions

In [3]:
instructions = "You provide help with tasks related to YouTube videos."

### define tool

In [4]:
@function_tool
def fetch_youtube_transcript(url: str) -> str:
    """
    Extract transcript with timestamps from a YouTube video URL and format it for LLM consumption
    
    Args:
        url (str): YouTube video URL
        
    Returns:
        str: Formatted transcript with timestamps, where each entry is on a new line
             in the format: "[MM:SS] Text"
    """
    # Extract video ID from URL
    video_id_pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
    video_id_match = re.search(video_id_pattern, url)
    
    if not video_id_match:
        raise ValueError("Invalid YouTube URL")
    
    video_id = video_id_match.group(1)
    
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        
        # Format each entry with timestamp and text
        formatted_entries = []
        for entry in transcript:
            # Convert seconds to MM:SS format
            minutes = int(entry['start'] // 60)
            seconds = int(entry['start'] % 60)
            timestamp = f"[{minutes:02d}:{seconds:02d}]"
            
            formatted_entry = f"{timestamp} {entry['text']}"
            formatted_entries.append(formatted_entry)
        
        # Join all entries with newlines
        return "\n".join(formatted_entries)
    
    except Exception as e:
        raise Exception(f"Error fetching transcript: {str(e)}")

### create agent

In [5]:
agent = Agent(
    name="YouTube Transcript Agent",
    instructions=instructions,
    tools=[fetch_youtube_transcript],
)

### main() function

In [6]:
async def main():
    input_items = []

    print("=== YouTube Transcript Agent ===")
    print("Type 'exit' to end the conversation")
    print("Ask me anything about YouTube videos!")

    while True:
        # Get user input
        user_input = input("\nYou: ").strip()
        input_items.append({"content": user_input, "role": "user"})
        
        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye!")
            break
            
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            agent,
            input=input_items,
        )

        async for event in result.stream_events(): # not all events are available at outset, hence the async
            # We'll ignore the raw responses event deltas
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            elif event.type == "agent_updated_stream_event":
                continue
            elif event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    print("\n-- Fetching transcript...")
                elif event.item.type == "tool_call_output_item":
                    input_items.append({"content": f"Transcript:\n{event.item.output}", "role": "system"})
                    print("-- Transcript fetched.")
                elif event.item.type == "message_output_item":
                    input_items.append({"content": f"{event.item.raw_item}", "role": "assistant"})
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

In [7]:
await main()
# try this video: https://youtu.be/ZaY5_ScmiFE

=== YouTube Transcript Agent ===
Type 'exit' to end the conversation
Ask me anything about YouTube videos!



You:  What is this video about? https://youtu.be/ZaY5_ScmiFE



Agent: 
-- Fetching transcript...
-- Transcript fetched.
The video is an introduction to a series on AI agents, presented by Shaw. It explores what AI agents are, their importance, and the differing definitions from leading organizations like OpenAI, Hugging Face, and Anthropic. The discussion includes:

1. **Key Features of AI Agents**:
   - Use of Large Language Models (LLMs).
   - Tool utilization (web search, code execution).
   - Autonomy and reasoning capabilities.

2. **Types of Agentic Systems**:
   - Level 1: LLMs with tools.
   - Level 2: LLM workflows with predefined steps.
   - Level 3: LLM in a feedback loop for real-world tasks.

3. **Design Patterns**:
   - Chaining, routing, parallelization, orchestrator-worker models, evaluator-optimizer loops.

4. **Value of AI Agents**: 
   - They interact with real-world data, improving over basic chatbots.
   - Test-time compute scaling enhances their response quality.

5. **Future Videos**:
   - The series will include building c


You:  Can you create timestamps for this video?



Agent: Certainly! Here are the timestamps for the video:

1. **Introduction** - 00:00
2. **What is an AI Agent?** - 00:05
3. **Definitions by Organizations** - 00:36
   - OpenAI - 00:41
   - Hugging Face - 00:51
   - Anthropic - 01:24
4. **Key Features of AI Agents** - 01:58
   - Use of LLMs - 02:10
   - Tool Usage - 02:35
   - Autonomy - 02:55
5. **Importance of AI Agents** - 03:36
6. **AI Agents vs. Chatbots** - 04:22
7. **Levels of Agentic Systems** - 06:36
   - Level 1: LLM + Tools - 06:54
   - Level 2: LLM Workflows - 11:01
   - Level 3: LLM in a Loop - 19:06
8. **Design Patterns in AI Agents** - 13:22
   - Chaining - 14:12
   - Routing - 14:44
   - Parallelization - 15:05
   - Orchestrator-Workers - 16:43
   - Evaluator-Optimizer - 17:59
9. **Future Series and Conclusion** - 22:52

Let me know if you need further details!




You:  Can you simplify? I only want 5-7 chapters/



Agent: Here's a simplified version with fewer chapters:

1. **Introduction to AI Agents** - 00:00
2. **Definitions and Key Features** - 00:36
3. **Importance and Use Cases** - 03:36
4. **AI Agents vs. Chatbots** - 04:22
5. **Levels of Agentic Systems** - 06:36
6. **Design Patterns** - 13:22
7. **Conclusion and Future Videos** - 22:52

Let me know if this works for you!




You:  exit



Goodbye!
